In [1]:
import yaml
import pandas as pd
from message_ix.utils import make_df
from message_ix.models import MESSAGE_ITEMS
from collections.abc import Mapping

<IPython.core.display.Javascript object>

In [18]:
with open('tech_data_01_update.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)
    
#this is something we need to get from the scenario
years = [*range(700,720,10)]
first_active_year = 700
vtg_years = [y for y in years if y >= first_active_year]
regions = ['Westeros', 'Westerlands']

In [19]:
# scenario.regions is the list of nodes in the scenario
default_cost_by_region = {reg: 1 for reg in regions}
cost_by_region = {
    tech: tech_data['model_data']
            .get(tech,{})
            .get('cost_by_region', default_cost_by_region)
    for tech in set(tech_data) - set(['model_data'])
}

cost_by_region

{'DACCS': {'Westeros': 1, 'Westerlands': 0.8}}

In [20]:
#par_dim = {'inv_cost':['node_loc'],'fix_cost':['node_loc'],'var_cost':['node_loc']}
parameters = {}
for tech in set(tech_data) - set(['model_data']):
    parameters.update({par: list(MESSAGE_ITEMS[par]['idx_names']) for par in set(tech_data[tech])})
data = {par: [] for par in list(parameters.keys())}

parameters

{'inv_cost': ['node_loc', 'technology', 'year_vtg']}

In [21]:
#Trial
for tech, par_dict in tech_data.items():
    if tech != 'model_data':
        for par, par_data in par_dict.items():
            print(tech,'-',par,'-',list(set(parameters.get(par))-set(['node_loc','technology']))) # this is important
            if not isinstance(par_data, Mapping):
                par_data = {'value': par_data, 'unit': '-'}
            data[par].append(
                make_df(
                    par,
                    technology=tech,
                    value=par_data['value'],
                    unit=par_data['unit'],
                )
            )

data = {k: pd.concat(v) for k, v in data.items()}

DACCS - inv_cost - ['year_vtg']


In [22]:
dummy = data['inv_cost']

data_expand = []
for tech, dims in tech_data['model_data'].items():
    for reg in regions:
        for y in vtg_years:
            data_expand.append(
                dummy.assign(
                    node_loc=reg,
                    year_vtg=y,
                    value=(dummy['value']
                           *dims.get('cost_by_region').get(reg)
                           *1
                          )
                    ))

pd.concat(data_expand)

,node_loc,technology,year_vtg,value,unit
0,Westeros,DACCS,700,2500.0,$/kW
0,Westeros,DACCS,710,2500.0,$/kW
0,Westerlands,DACCS,700,2000.0,$/kW
0,Westerlands,DACCS,710,2000.0,$/kW


In [16]:

#pd.DataFrame(years)

[700, 710]

In [ ]:
data = {k: pd.concat(v) for k, v in data.items()}

In [ ]:
with pd.ExcelWriter('tech_data.xlsx', engine='xlsxwriter', mode='w') as writer:
    for sheet_name, sheet_data in data.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
data